## NetCDF basics in Python

1. Read data from a NetCDF file
2. Create a simple contour plot

In [ ]:
import os, sys
import numpy as np
import pandas as pd
import netCDF4 as nc
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature


### Reading data from a NetCDF file

For this example, we will read monthly 2-meter temperature data from a netCDF file of ERA-Interim reanalysis (0.75 deg, 2000-2005).

In [ ]:
# Path to data file
datdir = '../sample-data/'
filename = 'erai.mon.t2m.2000-2005.nc'
filepath = datdir + filename
print(filepath)

# Open netCDF file object
f = nc.Dataset(filepath,'r')

# List file contents (variables, dimensions, global attributes)
print(f)

In [ ]:
# Read data into numpy arrays
lons = f.variables['longitude'][:] # [:] syntax stores data in numpy arrays
lats = f.variables['latitude'][:]
t2m  = f.variables['t2m'][:]


In [ ]:
# Retrieve attributes
t2m_units    = f.variables['t2m'].units
t2m_longname = f.variables['t2m'].long_name
t2m_missing  = f.variables['t2m'].missing_value
print('units:', t2m_units)


In [ ]:
# Print array size/shape
print(t2m.shape)

# Print size of lat/lon arrays...



In [ ]:
# Print data values
print(t2m)


### Time coordinate handling

Need to convert times from netcdf to a datetime array. 


In [ ]:
# Read time from netcdf file
time = f.variables['time'][:]
print(time)


In [ ]:
time_units = f.variables['time'].units
print(time_units)


**Option 1:** Read in times from netcdf file and convert it to a datetime object using the `num2date()` function. You will also need to retrieve in time.units attribute.

In [ ]:
# convert times to a datetime object
dates = nc.num2date(time, time_units)
print(dates)

**Option 2**: Create a pandas datetime object from scratch using `pd.date_range`

In [ ]:
dates_pd = pd.date_range(start='2000-01', end='2005-12', freq='M')
print(dates_pd)

### Simple arithmetic operations

In [ ]:
# Convert temperature to C
t2m_degC = t2m - 273.15
print(t2m_degC)


In [ ]:
# Average over the time dimension
t2m_mean = np.mean(t2m_degC, axis=0)
print(t2m_mean.shape)


In [ ]:
# Find min and max values of t2m_mean
tmax = t2m_mean.max()
tmin = t2m_mean.min()
print(tmin, tmax)


## Plotting

Create a filled contour plot of 2-meter temperature. We will use the cartopy and matplotlib packages. 


#### Map projection

When plotting with cartopy, you must specify the source coordinates of your data (**data coordinates**, `datacrs`) and the projection or coordinate system that you want to plot your data in (**map coordinates**, `mapcrs`). 

Always use "PlateCarree" if your data are in lat/lon coordinates. See link below for all available projections.

[Cartopy projection list](https://scitools.org.uk/cartopy/docs/latest/crs/projections.html#cartopy-projections)


In [ ]:
# Set your data coordinates
datacrs = ccrs.PlateCarree()
# Set the map projection
mapcrs = ccrs.PlateCarree()


In [ ]:
# Set up contour levels 
clevs = np.arange(-55, 41, 5)  # (start,stop,step)
print(clevs)


In [ ]:
# Create figure
fig = plt.figure(figsize=(7, 5))

# Add plot axes (an individual plot)
ax = fig.add_subplot(1, 1, 1, projection=mapcrs)

# Set up extent of the map [x0, x1, y0, y1]
ax.set_extent([lons.min(), lons.max(), lats.min(), lats.max()], crs=mapcrs)

# Add map features
ax.add_feature(cfeature.COASTLINE, edgecolor='0.9') #Grayscale colors can be set using 0 (black) to 1 (white)
ax.add_feature(cfeature.BORDERS, edgecolor='0.9')  

# Draw contour plot on the plot axes (ax)
p = ax.contourf(lons, lats, t2m_mean, transform=datacrs,
                levels=clevs,  # contour levels
                cmap='YlOrRd') # colormap

# Add colorbar
cbar = plt.colorbar(p, orientation='horizontal', label='deg C')

# Add plot title
plt.title('Average 2m Temperature (2000-2005)')

# Save to fig as png file
filename = 't2m-map.png'
plt.savefig(filename)

# Show figure
plt.show()

## Challenge Time!

1. Change the colormap of the plot. 

[Hint](https://matplotlib.org/tutorials/colors/colormaps.html)

2. Change the extent of the plot. 

[Hint](https://scitools.org.uk/cartopy/docs/v0.15/matplotlib/geoaxes.html#cartopy.mpl.geoaxes.GeoAxes.set_extent)